In [2]:

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from IPython.display import display, Markdown, Latex
import pandas as pd
from uncertainties import ufloat
from uncertainties import unumpy as unp
from scipy.optimize import curve_fit
from scipy import constants as const 

# PW7

## Experiment 1:

Grundsätzlich kann man Wärmeübertragung über drei Wege erreichen. Mit einem Übertragungsmedium kann es im Lichte der Wärmeübertragung entweder zu Konvektion oder Wärmeleitung kommen. Ersteres tritt mit bzw. durch Stoffbewegung auf, während letzteres auftritt, wenn es keine Stoffbewegung gibt.
In diesem ersten Experiment wird die Wärmeübertragung ohne Übertragungsmedium, also über Strahlung, näher betrachtet.
Das Experiment besteht aus einer Glühlampe mit monochromatischem Filter, welche an eine variable Spannungsversorgung angeschlossen ist, einer Fotozelle mit integriertem Strom-Spannungs-Wandler und einer optischen Bank, mit welcher der Abstand der Glühlampe und der Fotozelle ermittelt werden kann.
Bei leuchtender Glühlampe kommt es in der Fotozelle zu einem elektrischen Strom, welcher direkt proportional zur Beleuchtungsstärke E ist. Weil dieser sehr klein ist, verwendet man einen Strom-Spannungswandler, der eine Spannung U erzeugt, welche erneut direkt proportional zu Fotostrom ist. Somit ist die Spannung U proportional zur Beleuchtungsstärke E.
Nun wird die gemessene Spannung U auf den Abstand r graphisch aufgetragen. Das Verhältnis zwischen E und r ist gegeben durch $E = \frac{C I}{r^2}$
. Der Verlauf soll einem linearen Anstieg gleichen, wodurch man mittels eines Fits die Steigung a = CI ermitteln kann, wobei C eine Konstante ist und I die Lichtstärke. Führt man dies für zwei verschiedene Betriebsbedingungen durch, erhält man folgende Verhältnisse: 
$$
\frac{I_1}{I_2} = \frac{a_1}{a_2}
$$

Zum Schluss werden die dazugehörigen Leistungen P1 und P2, sowie die Strahlungstemperaturen der beiden Betriebsbedingungen ermittelt. Dafür bedient man sich folgenden Gleichungen:
$$
\frac{P_1}{P_2} = \frac{T_1^{4}}{T_2^{4}}
$$
$$
\ln\!\left(\frac{I_1}{I_2}\right)
= \frac{1}{T_1}
\left[
\left(\frac{P_1}{P_2}\right)^{1/4} - 1
\right]
\left(\frac{c h}{k \lambda}\right)
$$


In [3]:
#1.1

# bedingung 1
e_messwerte_1 = np.array([])
e_unsicherheiten_1 = np.array([])
e_werte_1 = unp.uarray[e_messwerte_1, e_unsicherheiten_1]

r_messwerte_1 = np.array([])
r_unsicherheiten_1 = np.array([])
r_werte_1 = unp.uarray[r_messwerte_1, r_unsicherheiten_1]
r_als_quotient_1 = 1/(r_werte_1)**2
r_quot_nom_1 = unp.nominal_values(r_als_quotient_1)

# bedingung 2 
e_messwerte_2 = np.array([])
e_unsicherheiten_2 = np.array([])
e_werte_2 = unp.uarray[e_messwerte_2, e_unsicherheiten_2]

r_messwerte_2 = np.array([])
r_unsicherheiten_2 = np.array([])
r_werte_2 = unp.uarray[r_messwerte_2, r_unsicherheiten_2]
r_als_quotient_2 = 1/(r_werte_2)**2
r_quot_nom_2 = unp.nominal_values(r_als_quotient_2)

x = r_als_quotient_1

def linear(x, m, b):
    return m*x + b

popt, pcov = curve_fit(linear, x, e_messwerte_1, sigma=e_unsicherheiten_1, absolute_sigma=True)
m, b = popt
dm, db = np.sqrt(np.diag(pcov))

x_fit = np.linspace(x.min(), x.max(), 200)
y_fit = linear(x_fit, m, b)

v = r_als_quotient_2

def linear2(v, c, d):
    return c*v + d

popt, pcov = curve_fit(linear2, v, e_messwerte_2, sigma=e_unsicherheiten_2, absolute_sigma=True)
c, d = popt
dc, dd = np.sqrt(np.diag(pcov))

v_fit = np.linspace(v.min(), v.max(), 200)
w_fit = linear2(v_fit, c, d)

# gegeneinander auftragen e bzw u anpassen
plt.errorbar(r_quot_nom_1, e_messwerte_1, yerr=e_unsicherheiten_1, fmt="o", capsize=4, label="Beleuchtungsstärke bei Lichttemperatur 1") #anpassen
plt.plot(x_fit, y_fit, label=f"Fit Lichttemperatur 1: E proportional zu U = {m:.3f} * 1/r^2 + {b:.3f}")

plt.errorbar(r_quot_nom_2, e_messwerte_2, yerr=e_unsicherheiten_2, fmt="o", capsize=4, label="Beleuchtungsstärke bei Lichttemperatur 2")
plt.plot(v_fit, w_fit, label=f"Fit Lichttemperatur 2: E proportional zu U = {c:.3f} * 1/r^2 + {d:.3f}")

plt.xlabel("Abstand in der Form 1/r^2 [mm]") #einheit?????
plt.ylabel("Beleuchtungsstärke bei der Bedingung 2 [lx]") #einheiten????
plt.title("Beleuchtungsstärke als Funktion des Abstands bei verschiedenen Lichttemperaturen")
plt.legend()
plt.grid(True)
plt.show()

print(f"Aus der Steigung des ersten Fits geht hervor, dass bei der Lichttemperatur 1 die Intensität I mal der Konstante C {m:.3f} +/- {dm:.3f} ist. Für die zweite Lichttemperatur beträgt C*I {c:.3f} +/- {dc:.3f}.")

verhältnis = m/c

print(f"Das Verhältnis der beiden Stromstärken zueinander beträgt {verhältnis}.")

p1/p2 = ufloat()
f =  const.Boltzmann * 680 #k mal lambda von 680 nm
g = const.c * const.h # lichtgeschwindigkeit mal plancksches wirkungsquantum

t1 = (((p1/p2)**(1/4))-1)/(np.log(verhältnis)*(f/g)) # hoffentlich ufloat?
t2 = ((1-((p1/p2)**(1/4)))*(g/f))/np.log(verhältnis) # hoffentlich auch ufloat?

print(f"Bei der ersten Betriebseinstellung hatte das Licht {t1}°C, bei der zweiten waren es {t2}°C.")



SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (1304238741.py, line 69)

## Experiment 2

Beim zweiten Experiment ist die Wärmeleitfähigkeit von Aluminium mittels thermografischer Auswertung von einem stationären Wärmestrom zu bestimmen. Das soll mit einer Wärmebildkamera geschehen.
Damit man es mit einem stationären Wärmestrom längs eines Metallstabes zu tun hat, muss jener zwischen einer Wärmequelle mit konstanter Heizleistung und Wärmesenke angebracht sein.
Nimmt man nun den Aufbau mit einer Wärmebildkamera auf, sieht man eine Verteilung verschiedener Temperaturen in Abhängigkeit vom Ort, welche folgender Relation entspricht:
$$ \frac{\Delta Q}{\Delta t} = - \lambda A \frac{\Delta T}{\Delta x} $$

Die Querschnittsfläche A wird mittels Schublehre ermittelt. Durch die Auswertung der Daten der Wärmebildkamera mit dem Programm „BMT“ erhält man die Temperaturverteilung T(x) und durch einen linearen Fit erhält man die Steigung ΔT/ Δx. Da in dem Aufbau ein Heizwiderstand R = 330Ω an einer Spannung U = 14V anliegt lässt sich $ \frac{\Delta Q}{\Delta t} = P = \frac{U^2}{R} $ als 0,60 W ermitteln und somit ist die Wärmeleitfähigkeit λ bestimmbar!

In [ ]:
# 2.

A = #fläche, gemessen
anstieg = #steigung, aus scidavis
t = 4 # K!!!! also auch C, konstanter wärmeunterschied

lambda_ = -(t/(anstieg*A)) #wärmeleitfähigkeit

print(f"Die Wärmeleitfähigkeit des Materials ist {lambda_} J/msK, dementsprechend handelt es sich bei der untersuchten Probe um Aluminium/Duraluminium.") # anpassen!!!!, differenz

# tabelle aus dem anleitungstext
"""Material λ (J m−1
s
−1 K−1
)
Diamant 2300
Kupfer 400
Aluminium 235
Duraluminium 170
Eisen (pur) 80
Stahl 50
Gusseisen 48
Edelstahl 1.4301 21
Glas 1
Wasser 0.54
Holz 0.15
Luft 0.024
Argon 0.016
Vakuumdämmplatte 0.005
"""



## Diskussion